### IMPORTANT INSTRUCTIONS
- The test cases only check type and structure of values returned by functions. So their correctness does not imply the correctness of your solutions.


## Modelling $e^x$
$y = f(x) = e^x$  

Approximate this function with a linear function of the form
$y = w_0 + w_1sin(\frac{x*\pi}{2}) + w_2sin(\frac{2*x*\pi}{2}) + w_3cos(\frac{x*\pi}{2}) + w_4cos(\frac{2*x*\pi}{2}) + \ldots$

in the range $x \in (1, 3)$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

## 1. Generate Data


### 1.1 
Write a function that takes a parameter $x$ (float or array of floats) and computes $y = f(x) = e^x$ (float if $x$ is a float, array of floats if $x$ is array of floats)  

In [2]:
def fx(x):
  '''
  Input:
    x : float (scalar) or np array
  Ouptut:
    y : float (scalar) or np array

  '''
  ### Write your code here
 
  y = np.exp(x)
  return y

In [3]:
"""Testing code"""
if __name__ == "__main__":
  y = fx(1.2)
  assert y/y == 1.0
  print("Test passed")

Test passed


### 1.2
Write a function that takes a parameter $n$ and generates $n$ random points in the range $(1, 3)$ $\rightarrow$ $x$ <br>
Using ```fx``` defined earlier, it also generates $yd$ corresponding to these n points and returns both $x$ and $yd$

In [4]:
def generate_points(n):
  '''
  Inputs:
    n : int, Number of random points
  
  Outputs:
    x : array of floats, random points in the range (1, 3)
    yd : array of floats, e^x for values in x
  '''
  ### Write your code here
  
  x = np.random.uniform(1,3,n)
  yd = fx(x)

  return x, yd



In [5]:
"""Testing code"""
if __name__ == "__main__":
  x, yd = generate_points(10)
  assert x.shape == (10, )
  assert yd.shape == (10, )
  print("Test passed")

Test passed


## 2. Training
We will now define a linear model to estimate the above function, and it will train with all the data.

### 2.1 
Define model
$y = w_0 + \sum_{d=1}^D w_d sin(\frac{d\pi x}{2}) + \sum_{d=D+1}^{2D} w_d cos(\frac{(d - D)\pi x}{2})$, here, $D$ is the degree of the model, say D=2 <br />

In matrix representation: y = Aw <br/>

In [6]:
def createA(x, D):
    '''
    Create the matrix A with degree D
    Input:
        x: np array of shape (N,)
        D: degree of the model
    Output:
        A: np matrix of shape (N,2D+1)
    '''
    ### Write your code here
    
    N = x.shape[0]
    A = np.ones((N,2*D+1))
    
    for d in range(1, D+1): 
        A[:,d] = np.sin(d*math.pi*x/2)
    
    
    for d in range(D+1,2*D+1): 
        A[:,d] = np.cos(d*math.pi*x/2)
        
    return A

In [7]:
"""Testing code"""
if __name__ == "__main__":
  x = np.arange(10)
  A = createA(x, 2)
  assert A.shape == (x.shape[0], 5)
  print("Test passed")

Test passed


### 2.2 Estimate weights
Train using least-square solution (pseudo-inverse) and min-norm solution that are used to train linear models.

In [8]:
def train_w(A, yd):
    '''
    Inputs:
        A: np array of shape (N,2D+1)
        yd: np array of shape (N,)
    Output:
        w: np array of shape (2D+1,)
    '''
    ### Write your code here
    
    N = yd.shape[0]
    D = A.shape[1]-1
    
    if N>D+1:
      w = np.linalg.inv(A.T.dot(A)).dot(A.T).dot(yd) 
    
    elif N<=D+1:
      w = A.T.dot(np.linalg.inv(A.dot(A.T))).dot(yd) 
    
    return w

In [9]:
"""Testing code"""
if __name__ == "__main__":
  yd = np.arange(10)
  A = np.random.random(50).reshape(10, 5)  
  w = train_w(A, yd)
  assert w.shape == (5, )
  print("Test passed")

Test passed


### 3 Estimate y from the model, given x and w

In [10]:
def predict_y(w, x):
    '''
    Inputs:
        w: np array of shape (2D+1,)
        x: np array of shape (N,)
    Outputs:
        y: np array of shape (N,); y=Aw
    '''
    ### Write your code here
    
    D = w.shape[0]-1
    A = createA(x,D//2)
    y = A.dot(w)
    
    return y

In [11]:
"""Testing code"""
if __name__ == "__main__":
  w = np.arange(5)
  x = np.arange(10)
  y = predict_y(w, x)
  assert y.shape[0] == x.shape[0]
  print("Test passed")

Test passed


### 4. Estimate Error
Find E as the mean squared error

In [12]:
def compute_mse(y, yd):
    '''
    Inputs:
        y: np array of shape (N,); y=Aw
        yd: np array of shape (N,); yd=f(x), ie., desired or true value
    Output:
        mse: float, mean squared error
    '''
    ### Write your code here
    
    mse = (y-yd).dot(y-yd)/y.shape[0]
    
    return mse

In [13]:
"""Testing code"""
if __name__ == "__main__":
  y = np.arange(3)
  yd = np.arange(30, 33)
  mse = compute_mse(y, yd)
  assert mse/mse == 1.0
  print("Test passed")

Test passed


## 5. Train and Test
Write a function that takes parameters N, D and:
- Generates N training points x, yd
- Train your linear model using x and yd
- Predict y (using the linear model you found above) for the training data x
- Compare y with yd to find the mean-squared error

### 5.1 Generate and Train

In [14]:
def trainModel(N, D):
    '''
    Inputs:
        N: number of samples
        D: degree of the model
    Outputs:
        x: np array of size (N,)
        y: np array of size (N,)
        yd: np array of size (N,)
        w: np array of size (2D+1,)
        mse: scalar float
    '''
    ### Write your code here
    
    x, yd = generate_points(N)
    A = createA(x, D)
    w = train_w(A, yd)
    y = predict_y(w, x)
    mse = compute_mse(y, yd)
    
    return x, y, yd, w, mse

In [15]:
"""Testing code"""
if __name__ == "__main__":
  x, y, yd, w, mse = trainModel(10, 2)
  assert x.shape[0] == 10
  assert y.shape[0] == 10
  assert yd.shape[0] == 10
  assert w.shape[0] == 5
  assert mse/mse == 1.0
  print("Tests passed")

Tests passed


### 5.2 Test
Write a function which takes parameters Ntest, w and does the following:

- Generates Ntest test points x, yd
- Estimate y using the linear model w 
- Compare y with yd to find the mean-squared error

In [16]:
def testModel(Ntest, w):
    '''
    Inputs:
        Ntest: number of test samples to be generated
        w: np array of size (2*D+1,)
    Outputs:
        x: np array of size (N,)
        y: np array of size (N,)
        yd: np array of size (N,)
        mse: scalar float
    '''
    ### Write your code here
    
    x, yd = generate_points(Ntest)
    D = w.shape[0]-1
    A = createA(x, D)
    print(A.shape)
    y = predict_y(w, x)
    mse = compute_mse(y, yd)
    
    return x, y, yd, mse

In [17]:
"""Testing code"""
if __name__ == "__main__":
  x, y, yd, mse = testModel(10, np.arange(5))
  assert x.shape[0] == 10
  assert y.shape[0] == 10
  assert yd.shape[0] == 10
  assert mse/mse == 1.0
  print("Tests passed")

(10, 9)
Tests passed
